
# 🧠 Fase 2 — Análise de Sentimentos (Português)  
**Projeto:** Mineração de Textos — Análise de Sentimentos  
**CRISP-DM:** Modeling / Evaluation  

Este notebook lê o CSV gerado na Fase 1, realiza **limpeza**, **detecção de idioma** e aplica um **modelo BERT-PT** para classificar **positivo/negativo/neutro**.  
Foi preparado para uso **autônomo** e para futura **integração com Flask** (sem `input()`), salvando a saída com **timestamp**.


In [ ]:

# ============================================================
# 1) Importações e parâmetros
# ============================================================
import os, re
import pandas as pd
import matplotlib.pyplot as plt
import emoji
from langdetect import detect, DetectorFactory
from transformers import pipeline
from glob import glob

DetectorFactory.seed = 0

INPUT_PATH = os.getenv("INPUT_PATH", "data/comentarios_coletados_*.csv")  # curinga tratado abaixo
OUTPUT_DIR = os.getenv("OUTPUT_DIR", "resultados")
HF_MODEL_PT = os.getenv("HF_MODEL_PT", "nlptown/bert-base-multilingual-uncased-sentiment")
print("✅ Bibliotecas carregadas.")

✅ Bibliotecas carregadas.


In [13]:

# ============================================================
# 2) Descoberta automática do último arquivo de coleta
# ============================================================
def ultimo_csv_coleta(pattern: str = "data/comentarios_coletados_*.csv") -> str:
    arquivos = sorted(glob(pattern))
    if not arquivos:
        raise FileNotFoundError("Nenhum arquivo de coleta encontrado. Execute a Fase 1 antes.")
    return arquivos[-1]

if "*" in INPUT_PATH:
    INPUT_PATH = ultimo_csv_coleta(INPUT_PATH)

print(f"📥 Lendo dados de: {INPUT_PATH}")
df = pd.read_csv(INPUT_PATH)
print(f"Registros carregados: {len(df)}")
display(df.head(5))

📥 Lendo dados de: data\comentarios_coletados_20251023_191025.csv
Registros carregados: 4371


,video_id,video_titulo,comentario,idioma,data_coleta
0,VKf6NF0OD5A,The new and improved YouTube Studio is here,Check out the entire series on how to use YouT...,en,2025-10-23
1,VKf6NF0OD5A,The new and improved YouTube Studio is here,Nice,ro,2025-10-23
2,VKf6NF0OD5A,The new and improved YouTube Studio is here,मेरा चैनल पर 100k सब्सक्राइबर है अभी तक सिल्व...,hi,2025-10-23
3,VKf6NF0OD5A,The new and improved YouTube Studio is here,❤❤❤❤❤,indefinido,2025-10-23
4,VKf6NF0OD5A,The new and improved YouTube Studio is here,Good❤❤❤❤ 0:25,so,2025-10-23


In [14]:

# ============================================================
# 3) Limpeza e normalização de texto
# ============================================================
def limpar_texto(texto: str) -> str:
    import re, emoji
    t = emoji.demojize(str(texto))
    t = re.sub(r"http\S+|www\.\S+", " ", t)
    t = re.sub(r"@[A-Za-z0-9_]+|#[A-Za-z0-9_]+", " ", t)
    t = re.sub(r"[^A-Za-zÀ-ÿ\s:]", " ", t)
    t = re.sub(r"\s+", " ", t).strip().lower()
    return t

df["comentario_limpo"] = df["comentario"].astype(str).apply(limpar_texto)
df = df[df["comentario_limpo"].str.len() > 3].copy()

print("🧹 Limpeza concluída.")
display(df[["video_titulo","comentario","comentario_limpo"]].head(5))

🧹 Limpeza concluída.


,video_titulo,comentario,comentario_limpo
0,The new and improved YouTube Studio is here,Check out the entire series on how to use YouT...,check out the entire series on how to use yout...
1,The new and improved YouTube Studio is here,Nice,nice
3,The new and improved YouTube Studio is here,❤❤❤❤❤,:red heart::red heart::red heart::red heart::r...
4,The new and improved YouTube Studio is here,Good❤❤❤❤ 0:25,good:red heart::red heart::red heart::red hear...
5,The new and improved YouTube Studio is here,Nice❤❤❤❤❤ 0:14,nice:red heart::red heart::red heart::red hear...


In [15]:

# ============================================================
# 4) Detecção de idioma e filtro (PT)
# ============================================================
def detectar_idioma_seguro(texto: str) -> str:
    try:
        return detect(texto)
    except Exception:
        return "indefinido"

df["idioma"] = df["comentario_limpo"].apply(detectar_idioma_seguro)
df_pt = df[df["idioma"] == "pt"].copy()

print(f"Total após limpeza: {len(df)} | Em português: {len(df_pt)}")
display(df_pt.head(5))

Total após limpeza: 4010 | Em português: 72


,video_id,video_titulo,comentario,idioma,data_coleta,comentario_limpo
34,VKf6NF0OD5A,The new and improved YouTube Studio is here,MR tanha video Editing,pt,2025-10-23,mr tanha video editing
142,VKf6NF0OD5A,The new and improved YouTube Studio is here,"Sim, o YouTube Studio é muito bom!🤩",pt,2025-10-23,sim o youtube studio é muito bom :star struck:
559,VKf6NF0OD5A,The new and improved YouTube Studio is here,Muito bom mesmo. Obrigado YouTube. Um prazer e...,pt,2025-10-23,muito bom mesmo obrigado youtube um prazer em ...
641,VKf6NF0OD5A,The new and improved YouTube Studio is here,"YouTube, tudo de bom, que seria do mundo sem v...",pt,2025-10-23,youtube tudo de bom que seria do mundo sem voc...
648,VKf6NF0OD5A,The new and improved YouTube Studio is here,Grade,pt,2025-10-23,grade


In [16]:

# ============================================================
# 5) Pipeline HuggingFace (PT-BR) — com batch
# ============================================================
sentiment_pt = pipeline(
    task="sentiment-analysis",
    model=HF_MODEL_PT,
    truncation=True,
    batch_size=32
)

def padronizar_label(lbl: str) -> str:
    m = {"POS": "positivo", "positive": "positivo",
         "NEG": "negativo", "negative": "negativo",
         "NEU": "neutro",   "neutral":  "neutro"}
    return m.get(lbl, lbl)

preds = sentiment_pt(df_pt["comentario_limpo"].tolist())
df_pt["sentimento"] = [padronizar_label(p["label"]) for p in preds]
df_pt["score"] = [p["score"] for p in preds]

print("✅ Análise concluída.")
display(df_pt[["video_titulo","comentario_limpo","sentimento","score"]].head(10))

OSError: pysentimiento/bert-base-portuguese-sentiment is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

In [ ]:

# ============================================================
# 6) Gráficos básicos
# ============================================================
if not df_pt.empty:
    dist = df_pt["sentimento"].value_counts().sort_index()
    plt.figure(figsize=(6,4))
    dist.plot(kind="bar")
    plt.title("Distribuição de Sentimentos (PT)")
    plt.xlabel("Sentimento")
    plt.ylabel("Quantidade")
    plt.tight_layout()
    plt.show()

    por_video = df_pt.groupby(["video_titulo","sentimento"]).size().unstack(fill_value=0)
    plt.figure(figsize=(10,5))
    por_video.plot(kind="bar", stacked=True)
    plt.title("Sentimentos por Vídeo (PT)")
    plt.xlabel("Vídeo")
    plt.ylabel("Quantidade")
    plt.tight_layout()
    plt.show()
else:
    print("Sem dados em PT para gráficos.")

In [ ]:

# ============================================================
# 7) Exportação com timestamp
# ============================================================
os.makedirs(OUTPUT_DIR, exist_ok=True)
ts = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
out_path = os.path.join(OUTPUT_DIR, f"comentarios_analisados_{ts}.csv")
df_pt.to_csv(out_path, index=False, encoding="utf-8-sig")

print(f"📦 Salvo em: {out_path} | Linhas: {len(df_pt)}")